In [1]:
import os
import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import biobss

/opt/homebrew/Caskroom/miniforge/base/envs/ArtHeart2/lib/python3.9/site-packages/plotly_resampler/aggregation/aggregators.py:24: UserWarning: Could not import lttbc; will use a (slower) python alternative.
  warnings.warn("Could not import lttbc; will use a (slower) python alternative.")
/opt/homebrew/Caskroom/miniforge/base/envs/ArtHeart2/lib/python3.9/site-packages/numba/cpython/hashing.py:482: UserWarning: FNV hashing is not implemented in Numba. See PEP 456 https://www.python.org/dev/peps/pep-0456/ for rationale over not using FNV. Numba will continue to work, but hashes for built in types will be computed using siphash24. This will permit e.g. dictionaries to continue to behave as expected, however anything relying on the value of the hash opposed to hash as a derived property is likely to not work as expected.
  warnings.warn(msg)
/opt/homebrew/Caskroom/miniforge/base/envs/ArtHeart2/lib/python3.9/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keywor

In [2]:

def load_data(dir):
    """
    Load data from the data folder
    """
    # Load data
    data = []
    order = []
    for file in os.listdir(dir):
        if file.endswith(".csv"):
            print(os.path.join(dir, file))
            df = pd.read_csv(os.path.join(dir, file),on_bad_lines='skip')
            if not df.empty:
                order.append(int(file[:-4]))
                data.append(df['DATA'])
    return data, order


try:
    Group1, Group1_order = load_data("/mnt/ArtHeart/Data/Group 1")
    Group2, Group2_order = load_data("/mnt/ArtHeart/Data/Group 2")
    Group3, Group3_order = load_data("/mnt/ArtHeart/Data/Group 3")
except:
    try:
        Group1, Group1_order = load_data("Data/Group 1")
        Group2, Group2_order = load_data("Data/Group 2")
        Group3, Group3_order = load_data("Data/Group 3")
    except:
        Group1, Group1_order = load_data("W:\PhD\ArtHeart\Data\Group 1")
        Group2, Group2_order = load_data("W:\PhD\ArtHeart\Data\Group 2")
        Group3, Group3_order = load_data("W:\PhD\ArtHeart\Data\Group 3")


Data/Group 1/6.csv
Data/Group 1/40.csv
Data/Group 1/7.csv
Data/Group 1/5.csv
Data/Group 1/4.csv
Data/Group 1/1.csv
Data/Group 1/3.csv
Data/Group 1/2.csv
Data/Group 1/37.csv
Data/Group 1/23.csv
Data/Group 1/22.csv
Data/Group 1/36.csv
Data/Group 1/20.csv
Data/Group 1/34.csv
Data/Group 1/35.csv
Data/Group 1/21.csv
Data/Group 1/25.csv
Data/Group 1/31.csv
Data/Group 1/19.csv
Data/Group 1/18.csv
Data/Group 1/30.csv
Data/Group 1/24.csv
Data/Group 1/32.csv
Data/Group 1/26.csv
Data/Group 1/27.csv
Data/Group 1/33.csv
Data/Group 1/16.csv
Data/Group 1/17.csv
Data/Group 1/29.csv
Data/Group 1/15.csv
Data/Group 1/14.csv
Data/Group 1/28.csv
Data/Group 1/10.csv
Data/Group 1/38.csv
Data/Group 1/39.csv
Data/Group 1/11.csv
Data/Group 1/13.csv
Data/Group 1/12.csv
Data/Group 1/9.csv
Data/Group 1/8.csv
Data/Group 2/6.csv
Data/Group 2/40.csv
Data/Group 2/7.csv
Data/Group 2/5.csv
Data/Group 2/4.csv
Data/Group 2/42.csv
Data/Group 2/46.csv
Data/Group 2/1.csv
Data/Group 2/47.csv
Data/Group 2/3.csv
Data/Group 2/44

In [4]:
def preprocess(data, windowsize = 600, fs = 24):
    """
    Preprocess data using BioBSS
    Returns: HRV measures for each specified window
    """
    # Preprocess data

    new_data = []
    
    for i in range(len(data)):
        windowed_data = []
        sig = data[i]

        sig = biobss.preprocess.filter_signal(sig,sampling_rate=fs,signal_type='PPG', method='bandpass')
        sig = biobss.preprocess.normalize_signal(sig)
        temp = biobss.preprocess.segment_signal(sig,sampling_rate=fs,window_size=windowsize,step_size=(windowsize/2))
        temp_hrv = []
        for j in range(len(temp)):
            sigs = temp[j]
            info=biobss.preprocess.peak_detection(sigs,fs,'peakdet',delta=0.01)

            locs_peaks=info['Peak_locs']
            peaks=sig[locs_peaks]
            locs_onsets=info['Trough_locs']
            onsets=sig[locs_onsets] 

            info=biobss.ppgtools.peak_control(sig=sigs, peaks_locs=locs_peaks, troughs_locs=locs_onsets)

            locs_peaks=info['Peak_locs']
            peaks=sig[locs_peaks]
            locs_onsets=info['Trough_locs']
            onsets=sig[locs_onsets]

            length = len(sigs)/fs
            hrv = biobss.hrvtools.get_hrv_features(sampling_rate=fs, signal_length=length, input_type='peaks',peaks_locs=locs_peaks)
            hrv = pd.DataFrame(hrv.values(), index=hrv.keys())
            temp_hrv.append(hrv)
        temp_hrv = pd.concat(temp_hrv, axis=1)
        new_data.append(temp_hrv)

    return new_data

Group1 = preprocess(Group1)
Group2 = preprocess(Group2)
Group3 = preprocess(Group3)

/opt/homebrew/Caskroom/miniforge/base/envs/ArtHeart2/lib/python3.9/site-packages/biobss/ppgtools/ppg_filter.py:43: UserWarning: Default parameters will be used for filtering. 2th order bandpass filter with f1=0.5 Hz and f2=5 Hz.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ArtHeart2/lib/python3.9/site-packages/biobss/ppgtools/ppg_filter.py:43: UserWarning: Default parameters will be used for filtering. 2th order bandpass filter with f1=0.5 Hz and f2=5 Hz.
  warnings.warn(


KeyboardInterrupt: 

/opt/homebrew/Caskroom/miniforge/base/envs/ArtHeart2/lib/python3.9/site-packages/biobss/ppgtools/ppg_filter.py:43: UserWarning: Default parameters will be used for filtering. 2th order bandpass filter with f1=0.5 Hz and f2=5 Hz.
  warnings.warn(


Exception: **ERROR** window_size must be smaller than the length of signal. Make sure you entered window size in seconds.